In [1]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import datetime
import time

In [2]:
def check_symbol (stock_df ,label) :
    test_dr = pd.DataFrame()
    for i in range (len(stock_df)):
        if len(stock_df.iloc[i][label]) == 4:
            test_dr = test_dr.append(stock_df[i:i+1]).reset_index(drop=True)
    return test_dr

In [3]:
def extract(Stock_list, whole_df , label):
    start = 0
    end = 0
    point = Stock_list[0]
    for i in range (len(whole_df)):
        if whole_df.iloc[i][label] >= str(point):
            #print (whole_df.iloc[i][label] , i)
            if start != 0 :
                end = i
                break
            point = Stock_list[len(Stock_list)-1]
            start = i
    test =  pd.DataFrame(whole_df[start:end+1])
    return test

In [4]:
def find_index(Stock_list, extract, label,  null) :
    institutional = pd.DataFrame()
    for i in range (len(Stock_list)):
        a = extract[extract[label] == str(Stock_list[i])].index
        if len(a)== 1  :
            institutional=institutional.append(extract.iloc[a])
        elif len(a) == 0 :
            institutional=institutional.append(null)
    return institutional.reset_index(drop=True)

In [5]:
def devide(list1 , list2) :
    out_come = list()
    for i in range (len(list1)):
        if type (list1[i]) != float and  int(list2[i].replace(",", "")) >0 :
            x = format(int(list1[i].replace(",", "")) / int(list2[i].replace(",", "")) ,'.3f')
        else :
            x = 0
        out_come.append(x)
    return out_come

In [6]:
Stock_list = pd.read_csv('name.csv')["name"]
date_list = pd.read_csv('date.csv')['date'].tolist()
whole_data = pd.DataFrame()
Stock_index = pd.read_csv('Market_index.csv')

i=0
today = datetime.date.today()
day = str (today-datetime.timedelta(days=i+553)).replace("-", "")
day

In [7]:
# generate date
i = 0
today = datetime.date.today()
day = str (today-datetime.timedelta(days=i)).replace("-", "")
record_day =  max(date_list) 
sleep_download_time = 3

while int(day) > record_day  :
#while i < 4000  :
    # get basic data
    
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + day + '&type=ALL')
    if len(r.text) < 100 :
        print (day ," is not work day!")
        time.sleep(sleep_download_time)
        i+=1
        day = str (today-datetime.timedelta(days=i)).replace("-", "")
        continue
        
    label_ = "證券代號"
    df = pd.read_csv(StringIO(r.text.replace("=", "")),  header=["證券代號" in l for l in r.text.split("\n")].index(True)-1).drop(['Unnamed: 16'],axis = 1)
    df = df.dropna().reset_index(drop=True)
    time.sleep(sleep_download_time)
    
    # process the basic data by name list
    base_df = extract(Stock_list , df , label_).reset_index(drop=True)
    stock_df = find_index(Stock_list, base_df, label_,  df[-1:])
    
    ## get institution data
    r = requests.get('https://www.twse.com.tw/fund/T86?response=csv&date=' + day + '&selectType=ALL')
    r = r.text.replace("=", "")
    df2 = pd.read_csv(StringIO(r.replace("  \n", "")),  header=["證券代號" in l for l in r.split("\r\n")].index(True))
    df2.sort_values(by='證券代號' , ascending= True, axis =0 , inplace=True)
    df2 = df2.reset_index(drop=True).drop(['Unnamed: 19'],axis = 1)
    
    institutional = extract(Stock_list , df2 , label_).reset_index(drop=True)
    institutional = find_index(Stock_list, institutional, label_,  df2[-1:])
    time.sleep(sleep_download_time)
    ## process margin data
    r = requests.get('https://www.twse.com.tw/exchangeReport/MI_MARGN?response=csv&date=' + day + '&selectType=ALL')
    r = r.text.replace("=", "")
    df3 = pd.read_csv(StringIO(r), header=["股票代號" in l for l in r.split("\n")].index(True)).drop(['Unnamed: 16'],axis = 1)

    label_ = "股票代號"
    margin = extract(Stock_list, df3 , label_).reset_index(drop=True)
    margin = find_index(Stock_list, margin, label_,  df3[-1:])
    ratio = devide(margin['今日餘額.1'], margin['今日餘額'] )
    margin ['ratio'] = ratio 
    time.sleep(sleep_download_time)
    
    # merge to basic df
    stock_df['融資買'] = margin['買進']
    stock_df['融資賣'] = margin['賣出']
    stock_df['融資餘'] = margin['今日餘額']
    stock_df['融卷買'] = margin['買進.1']
    stock_df['融卷賣'] = margin['賣出.1']
    stock_df['融卷餘'] = margin['今日餘額.1']
    stock_df['ratio'] = margin ['ratio']

    stock_df['外資進出'] = institutional['外陸資買賣超股數(不含外資自營商)']
    stock_df['外資進'] = institutional['外陸資買進股數(不含外資自營商)']
    stock_df['外資出'] = institutional['外陸資賣出股數(不含外資自營商)']
    stock_df['投信進出'] = institutional['投信買賣超股數']
    stock_df['投信進'] = institutional['投信買進股數']
    stock_df['投信出'] = institutional['投信賣出股數']
    stock_df['自營進出'] = institutional['自營商買賣超股數(自行買賣)']
    stock_df['自營進'] = institutional['自營商買進股數(自行買賣)']
    stock_df['自營出'] = institutional['自營商賣出股數(自行買賣)']
    stock_df['避險進出'] = institutional['自營商買賣超股數(避險)']
    stock_df['避險進'] = institutional['自營商買進股數(避險)']
    stock_df['避險出'] = institutional['自營商賣出股數(避險)']
    stock_df['三大'] = institutional['三大法人買賣超股數']
    
    #stock_df  = stock_df.dropna().reset_index(drop=True)
    
    ## add date label
    date = [day for i in range(len(stock_df))]
    stock_df ["date"] = date
    whole_data = whole_data.append(stock_df)
    date_list.append(int(day))
    
    # for market index
    index_set = ['發行量加權股價指數' , '未含電子指數' , '電子類兩倍槓桿指數' ]
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+ day +'&type=IND')
    df = pd.read_csv(StringIO(r.text.replace("價格指數(臺灣證券交易所)\"\r\n", "")),  header=["指數" in l for l in r.text.split("\r\n")].index(True)).drop(['漲跌(+/-)','特殊處理註記','Unnamed: 6'],axis = 1)
    df1 = df.dropna()
    df1.columns = ['Index','收盤','漲跌','漲跌%']
    index = pd.DataFrame()
    for x in index_set :
        index = index.append(df1[df1.Index == x])
    index['date'] = [int(day) for x in range (len(index))]
    Stock_index = Stock_index.append(index)
    time.sleep(sleep_download_time)    
    print (day)
    
    i+=1
    day = str (today-datetime.timedelta(days=i)).replace("-", "")

20210504


In [8]:
date = pd.DataFrame()
date['date'] = date_list
date = date.drop_duplicates()
date = date.sort_values(by= 'date',ascending=False)
date.to_csv('date.csv', index = False)

In [9]:
date

,date
537,20210504
0,20210503
1,20210429
2,20210428
3,20210427
...,...
532,20190221
533,20190220
534,20190219
535,20190218


In [10]:
drop = whole_data.dropna().reset_index(drop=True)
whole_data = drop.drop(['漲跌(+/-)', '證券名稱' ,'最後揭示買價', '最後揭示買量', '最後揭示賣價', '最後揭示賣量'],axis = 1)

In [11]:
for l in whole_data.columns : 
    list_ = []
    for i in range(len(whole_data)) :
        k = whole_data[l][i]
        if type(k) == str :
            try :
                list_.append(float (k.replace(",", "")))
            except:
                print(l , i , k)
                list_.append(0)
                continue
        else : 
            list_.append(float(k))
    whole_data[l] = list_

In [12]:
Stock_index = Stock_index.drop_duplicates()
Stock_index = Stock_index.reset_index(drop=True)
Stock_index

,Index,收盤,漲跌,漲跌%,date
0,發行量加權股價指數,16211.7,264.85,1.66,20210303
1,未含電子指數,16115,273.49,1.73,20210303
2,電子類兩倍槓桿指數,40860.8,1287.77,3.25,20210303
3,發行量加權股價指數,15946.9,6.92,-0.04,20210302
4,未含電子指數,15841.5,60.53,-0.38,20210302
...,...,...,...,...,...
1559,未含電子指數,19061.2,6.78,-0.04,20210429
1560,電子類兩倍槓桿指數,43456.5,11.19,0.03,20210429
1561,發行量加權股價指數,"16,933.78",288.57,-1.68,20210504
1562,未含電子指數,"18,437.18",530.05,-2.79,20210504


In [13]:
for l in ['收盤','漲跌','漲跌%'] : 
    list_ = []
    for i in range(len(Stock_index)) :
        k = Stock_index[l][i]
        if type(k) == str :
            try :
                list_.append(float (k.replace(",", "")))
            except:
                print(l , i , k)
                list_.append(0)
                continue
        else : 
            list_.append(float(k))
    Stock_index[l] = list_

In [14]:
whole_data = whole_data [whole_data['開盤價'] > 0]
whole_data

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date
0,1102.0,21282563.0,8415.0,1.039808e+09,49.60,49.90,48.25,48.55,1.05,11.08,...,3000.0,812000.0,1370000.0,1620000.0,250000.0,450000.0,565000.0,115000.0,-2756000.0,20210504.0
1,1103.0,5613651.0,2571.0,1.375425e+08,25.20,25.70,23.50,24.15,0.85,8.81,...,0.0,0.0,16000.0,18000.0,2000.0,0.0,0.0,0.0,1594000.0,20210504.0
2,1104.0,3816821.0,1914.0,9.219046e+07,25.05,25.45,23.30,23.95,1.05,12.54,...,0.0,0.0,-55000.0,5000.0,60000.0,0.0,0.0,0.0,878000.0,20210504.0
3,1108.0,4055547.0,1524.0,6.246711e+07,16.75,16.75,14.70,15.50,0.80,15.98,...,0.0,0.0,3000.0,3000.0,0.0,0.0,0.0,0.0,1427000.0,20210504.0
4,1109.0,2786808.0,1188.0,7.647747e+07,28.85,29.00,26.25,28.00,0.50,9.52,...,0.0,0.0,9248.0,53248.0,44000.0,0.0,0.0,0.0,549248.0,20210504.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,9944.0,528679.0,358.0,1.064603e+07,21.00,21.00,19.50,20.40,0.65,29.57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-122000.0,20210504.0
911,9945.0,18891286.0,9426.0,9.962220e+08,53.70,54.60,49.80,52.20,0.90,9.14,...,0.0,0.0,63474.0,79597.0,16123.0,71000.0,117000.0,46000.0,-261451.0,20210504.0
912,9946.0,677674.0,368.0,1.079496e+07,16.40,16.40,15.45,15.85,0.60,27.81,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-135000.0,20210504.0
913,9955.0,2586477.0,1382.0,5.394081e+07,22.35,22.80,19.95,20.20,1.95,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,293000.0,20210504.0


In [15]:
previous = pd.read_csv('clear.csv')
previous = previous.drop_duplicates()
previous 

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date
0,1102.0,11829848.0,4783.0,587379311.0,49.75,49.95,49.5,49.60,0.35,11.32,...,0.0,138000.0,451000.0,607000.0,156000.0,96000.0,117000.0,21000.0,466900.0,20210503.0
1,1103.0,5701066.0,2453.0,144768008.0,26.00,26.00,25.0,25.00,1.00,9.12,...,9000.0,0.0,74000.0,115000.0,41000.0,0.0,0.0,0.0,1122000.0,20210503.0
2,1104.0,3565899.0,1571.0,90727253.0,26.10,26.10,25.0,25.00,1.10,13.09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,869000.0,20210503.0
3,1108.0,2617699.0,1129.0,43493892.0,16.95,17.10,16.3,16.30,0.60,16.80,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,564000.0,20210503.0
4,1109.0,1310701.0,736.0,37822927.0,29.05,29.55,28.3,28.50,0.55,9.69,...,0.0,0.0,60067.0,60067.0,0.0,0.0,0.0,0.0,144367.0,20210503.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457872,9943.0,190438.0,168.0,12812017.0,67.50,67.50,67.2,67.20,0.20,11.18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-52000.0,20190920.0
457873,9945.0,2752245.0,1096.0,116365981.0,42.60,42.60,42.1,42.10,0.50,4.38,...,0.0,81000.0,40000.0,40000.0,0.0,-56000.0,0.0,56000.0,-684265.0,20190920.0
457874,9946.0,1184358.0,469.0,27781296.0,23.50,23.60,23.3,23.60,0.10,5.58,...,0.0,0.0,-78000.0,12000.0,90000.0,0.0,0.0,0.0,107000.0,20190920.0
457875,9955.0,538156.0,334.0,10280286.0,18.80,19.20,18.8,19.15,0.35,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44000.0,20190920.0


In [16]:
K = whole_data.append(previous).reset_index(drop=True)
K

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date
0,1102.0,21282563.0,8415.0,1.039808e+09,49.60,49.90,48.25,48.55,1.05,11.08,...,3000.0,812000.0,1370000.0,1620000.0,250000.0,450000.0,565000.0,115000.0,-2756000.0,20210504.0
1,1103.0,5613651.0,2571.0,1.375425e+08,25.20,25.70,23.50,24.15,0.85,8.81,...,0.0,0.0,16000.0,18000.0,2000.0,0.0,0.0,0.0,1594000.0,20210504.0
2,1104.0,3816821.0,1914.0,9.219046e+07,25.05,25.45,23.30,23.95,1.05,12.54,...,0.0,0.0,-55000.0,5000.0,60000.0,0.0,0.0,0.0,878000.0,20210504.0
3,1108.0,4055547.0,1524.0,6.246711e+07,16.75,16.75,14.70,15.50,0.80,15.98,...,0.0,0.0,3000.0,3000.0,0.0,0.0,0.0,0.0,1427000.0,20210504.0
4,1109.0,2786808.0,1188.0,7.647747e+07,28.85,29.00,26.25,28.00,0.50,9.52,...,0.0,0.0,9248.0,53248.0,44000.0,0.0,0.0,0.0,549248.0,20210504.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458787,9943.0,190438.0,168.0,1.281202e+07,67.50,67.50,67.20,67.20,0.20,11.18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-52000.0,20190920.0
458788,9945.0,2752245.0,1096.0,1.163660e+08,42.60,42.60,42.10,42.10,0.50,4.38,...,0.0,81000.0,40000.0,40000.0,0.0,-56000.0,0.0,56000.0,-684265.0,20190920.0
458789,9946.0,1184358.0,469.0,2.778130e+07,23.50,23.60,23.30,23.60,0.10,5.58,...,0.0,0.0,-78000.0,12000.0,90000.0,0.0,0.0,0.0,107000.0,20190920.0
458790,9955.0,538156.0,334.0,1.028029e+07,18.80,19.20,18.80,19.15,0.35,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44000.0,20190920.0


In [17]:
K.columns

Index(['證券代號', '成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差',
       '本益比', '融資買', '融資賣', '融資餘', '融卷買', '融卷賣', '融卷餘', 'ratio', '外資進出', '外資進',
       '外資出', '投信進出', '投信進', '投信出', '自營進出', '自營進', '自營出', '避險進出', '避險進', '避險出',
       '三大', 'date'],
      dtype='object')

In [18]:
K.to_csv('clear.csv', index = False)
Stock_index.to_csv('Market_index.csv', index = False)